---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [5]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [149]:
def date_sorter():
    ## Import Libraries ##
    
    import numpy as np
    
    
    ## Process data ##
    
    # Create list of regex for different formats with non-reporting groups:
    #regex_list = [
    #    r'(?:\d{1,2}(?:/|-)\d{1,2}(?:/|-)\d{2,4})',
    #    r'(?:[A-Z]\w+\.?\s\d{1,2},?\s\d{2,4})',
    #    r'(?:\d{1,2}\s[A-Z]\w+,?\.?\s\d{2,4})',
    #    #r'(?:[A-Z]\w+\s\d{1,2}(?:st|nd|rd|th),\s\d{2,4})',#this case is not in the data
    #    r'(?:[A-Z]\w+,?\s\d{4})',
    #    r'(?:\d{1,2}/\d{2,4})',
    #    r'(?:\d{4})'
    #]
    
    # Create Series with extracted dates:
    #dates = df.str.extract('(' + '|'.join(regex_list) + ')')
    
    # Create list of regex to extract day, month and year from text:
    regex_list = [
        r'(?P<month>\d{1,2})(?:/|-)(?P<day>\d{1,2})(?:/|-)(?P<year>\d{2,4})', # 1st format
        r'(?P<month>[A-Z]\w+)\.?\s(?P<day>\d{1,2}),?\s(?P<year>\d{2,4})', # 2nd format
        r'(?P<day>\d{1,2})\s(?P<month>[A-Z]\w+),?\.?\s(?P<year>\d{2,4})', # 3rd format
        r'(?P<month>[A-Z]\w+),?\s(?P<year>\d{4})', # 5th format
        r'(?P<month>\d{1,2})/(?P<year>\d{2,4})', # 6th format
        r'(?P<year>\d{4})' # 7th format
    ]
    
    # Create list to store extractions:
    extractions = []
    # Loop over regex list:
    for regex in regex_list:
        extractions.append(df.str.extract(regex))
    
    # Create missing columns for some extractions:
    extractions[3]['day'] = 1
    extractions[4]['day'] = 1
    # Convert series to dataframe then fill:
    extractions[5] = extractions[5].to_frame()
    extractions[5]['day'] = 1
    extractions[5]['month'] = 1
    
    # Merge results from first to last data:
    i = 0
    while i < len(extractions)-1: #stop at last but one
        
        # MERGE LOGIC HERE!!!!!
        
        # Update counter
        i += 1
    
    # Extract day, month and year from these strings:
    #ext_dates = dates.str.extract(r'(?P<month>\d{1,2})(?:/|-)(?P<day>\d{1,2})(?:/|-)(?P<year>\d{2,4})')
    #years = dates.str.extract(r'(?P<month>[A-Z]\w+)\.?\s(?P<day>\d{1,2}),?\s(?P<year>\d{2,4})')
    
    # Merge results:
    #final = pd.merge(ext_dates, years, how='outer', left_index=True, right_index=True)
    #final['month'] = final['month_x'].fillna(final['month_y'])
    #final.drop(['month_x', 'month_y'], axis=1, inplace=True)
    
    # Convert index to column:
    
    # Sort based on date:
    
    
    # Return only sorted indices:
    return extractions#dates #final

# Testing:
test = date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [150]:
test[5].head()

,year,day,month
0,NaN,1,1
1,NaN,1,1
2,NaN,1,1
3,NaN,1,1
4,NaN,1,1


In [137]:
test[0]['month'].isnull().sum()

72

In [113]:
test['month'].isnull().nonzero()[0]

array([125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137,
       138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150,
       151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163,
       164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176,
       177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189,
       190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202,
       203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,
       216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228,
       229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241,
       242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254,
       255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267,
       268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293,
       294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 30

In [109]:
test.iloc[455]

'1984'

In [110]:
test.str.date

AttributeError: Can only use .dt accessor with datetimelike values